In [ ]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dynamo.preprocessing import Preprocessor
import dynamo as dyn
import palantir
%matplotlib inline
import matplotlib.colors as mcolors
import scipy.sparse as sp

In [ ]:
counts_data = pd.read_csv("counts_data.csv", index_col=0)
counts_data.columns = counts_data.columns.str.replace('^SeuratProject', '', regex=True)
counts_data = counts_data.T
metadata = pd.read_csv("metadata.csv", index_col=0)
metadata.index = metadata.index.str.replace('^SeuratProject', '', regex=True)

adata = ad.AnnData(X=counts_data.values, obs=metadata, var=pd.DataFrame(index=counts_data.columns))
adata = adata[adata.obs['sample'] == 222107]

types = pd.read_csv('adpkd_clustering.csv')
types['Barcode'] = types['Barcode'] + '_2'
types.index = types['Barcode']
types = types.drop('Barcode', axis = 1)
adata.obs['cell_types'] = types['Kidney group 222107 analysis']

adata = adata[adata.obs['cell_types'].isna() == False]

In [ ]:
adata.X = adata.X.astype(float)
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
dm_res = palantir.utils.run_diffusion_maps(adata, n_components=5)

In [ ]:
ms_data = palantir.utils.determine_multiscale_space(adata)

In [ ]:
imputed_X = palantir.utils.run_magic_imputation(adata)
palantir.plot.plot_diffusion_components(adata)
plt.show()

In [ ]:
imputed_X = palantir.utils.run_magic_imputation(adata)

In [ ]:
terminal_states = palantir.utils.find_terminal_states(adata, celltypes = ['DCT','LOH', 'COI'], celltype_column = 'cell_types')

In [ ]:
palantir.plot.highlight_cells_on_umap(adata, terminal_states)
plt.show()

In [ ]:
start_cell = 'TGTCTACTAGATTGCC-1_2'
pr_res = palantir.core.run_palantir(
    adata, start_cell, num_waypoints=500, terminal_states=terminal_states)

In [ ]:
palantir.plot.plot_palantir_results(adata)
plt.show()

In [ ]:
adata.layers['M_s']=adata.X

if 'M_s' in adata.layers and not sp.issparse(adata.layers['M_s']):
    adata.layers['M_s'] = sp.csr_matrix(adata.layers['M_s'])

In [ ]:
dyn.tl.pseudotime_velocity(adata, pseudotime='palantir_pseudotime')
dyn.pl.streamline_plot(adata, color=['cell_types'], basis='umap', show_legend='on data', figsize=(9, 12), alpha = 0.3, pointsize=1.0, dpi=600, show_arrowed_spines = True)